# Messaging services

- store and pull messages
- manage read receipts

In [ ]:
//| export

import type { Database } from "tinychat/db.ts";
import { Record as Message } from "tinychat/api/types/chat/tinychat/core/message.ts";
import {
  MessageView,
  validateMessageView,
} from "tinychat/api/types/chat/tinychat/server/defs.ts";
import { getTimeus, removeNulls } from "tinychat/utils.ts";
import { DeleteMessageRecord } from "tinychat/firehose.ts";

export class MessageCursor {
  constructor(public timestamp: string, public direction: "past" | "future") {}

  public static fromString(base64Str: string): MessageCursor {
    const decoded = atob(base64Str);
    const [timestamp, direction] = decoded.split(":");
    return new MessageCursor(timestamp, direction as "past" | "future");
  }

  public toString(): string {
    return btoa(`${this.timestamp}:${this.direction}`);
  }
}

export class Messaging {
  constructor(protected db: Database) {}

  public deleteMessage({ uri }: DeleteMessageRecord) {
    this.db
      .prepare(`UPDATE messages SET deleted_at = :time, text = '<deleted>' WHERE uri = :uri`)
      .run({ uri, time: new Date().toISOString() });
  }

  public markAllMessagesAsRead({
    channel,
    server,
    user,
  }: {
    channel: string;
    server: string;
    user: string;
  }) {
    this.db
      .prepare(
        `INSERT OR REPLACE INTO read_receipts (channel, server, user, time_us) VALUES (:channel, :server, :user, :time)`,
      )
      .run({ channel, user, server, time: getTimeus() });
  }

  public receiveMessage({
    m,
    uri,
    sender,
    time_us,
  }: {
    m: Message;
    uri: string;
    sender: string;
    time_us: string;
  }) {
    this.db
      .prepare(
        `
      INSERT INTO messages (uri, channel, server, text, sender, created_at, time_us) VALUES (
        :uri, :channel, :server, :text, :sender, :created_at, :time_us
      )`,
      )
      .run({
        uri,
        channel: m.channel,
        server: m.server,
        text: m.text,
        sender,
        created_at: m.createdAt,
        time_us: time_us,
      });
  }

  public getMessages({
    server,
    channel,
    uri,
    cursor,
    limit,
    sort = "latest",
  }: {
    server?: string;
    channel?: string;
    uri?: string;
    cursor?: string;
    limit?: number;
    sort?: "latest" | "chronological";
  }): {
    messages: MessageView[];
    prevCursor?: string;
    nextCursor?: string;
  } {
    interface Message {
      uri: string;
      channel: string;
      server: string;
      text: string;
      createdAt: string;
      time_us: string;
      // user
      did: string;
      handle: string;
      displayName: string;
      avatar?: string;
      description?: string;
    }

    if (!channel && !server && !uri) {
      return {
        messages: [],
      };
    }

    const parsedCursor = cursor && MessageCursor.fromString(cursor);
    const cursorWhere = (c: MessageCursor) =>
      c.direction === "past" ? "time_us < :cursor" : "time_us > :cursor";
    let results: Message[] = [];

    if (uri) {
      results = this.db
        .prepare(`SELECT * FROM message_view WHERE uri = :uri`)
        .all<Message>(Object.assign({ uri }));
    } else {
      results = this.db
        .prepare(
          `SELECT * FROM message_view
      WHERE channel = :channel AND server = :server ${
            parsedCursor ? `AND ${cursorWhere(parsedCursor)}` : ""
          } ORDER BY ${
            sort === "chronological" ? "time_us ASC" : "time_us DESC"
          } LIMIT :limit`,
        )
        .all<Message>(
          Object.assign(
            { server, channel, limit: limit || 10 },
            parsedCursor ? { cursor: parsedCursor.timestamp } : {},
          ),
        );
    }

    const messages: MessageView[] = results
      .map((m: Message) => ({
        uri: m.uri,
        channel: m.channel,
        server: m.server,
        text: m.text,
        createdAt: m.createdAt,
        ts: m.time_us,
        sender: {
          did: m.did,
          handle: m.handle,
          displayName: m.displayName,
          avatar: m.avatar,
          description: m.description,
        },
      }))
      .map(removeNulls)
      .map((m) => {
        const v = validateMessageView(m);
        if (!v.success) {
          console.error("Failed to validate message view", v);
        }
        // @ts-ignore yolo
        return v.value;
      })
      .filter((m) => m) as MessageView[];

    if (sort === "latest") {
      return Object.assign(
        {
          messages,
        },
        messages.length === limit
          ? {
            prevCursor: new MessageCursor(
              messages[messages.length - 1].ts,
              "past",
            ).toString(),
          }
          : {},
        cursor
          ? {
            nextCursor: new MessageCursor(
              messages[0].ts,
              "future",
            ).toString(),
          }
          : {},
      );
    }

    // chronological ordering
    // need to check if we have older messages for the prev cursor

    const hasPreviousMessages = messages.length !== 0 && this.db
          .prepare(
            `SELECT uri FROM message_view
          WHERE channel = :channel AND server = :server AND time_us < :time_us
          ORDER BY time_us DESC LIMIT :limit`,
          )
          .all<Message>({
            channel,
            server,
            time_us: messages[0].ts,
            limit: 1,
          }).length > 0;

    return Object.assign(
      {
        messages,
      },
      messages.length === limit
        ? {
          nextCursor: new MessageCursor(
            messages[messages.length - 1].ts,
            "future",
          ).toString(),
        }
        : {},
      hasPreviousMessages
        ? {
          prevCursor: new MessageCursor(messages[0].ts, "past").toString(),
        }
        : {},
    );
  }
}

In [ ]:
// import { j } from "@jurassic/jurassic";
// j.initialize("messaging.ipynb");
// await j`
// rebuild this so channels list is pulled in separate subquery instead of a join

// SELECT
//         s.uri,
//         s.name,
//         s.creator,
//         json_group_array(
//           json_object(
//             'uri', c.uri,
//             'name', c.name,
//             'last_message_read_time_us', c.last_message_read_time_us,
//             'latest_message_received_time_us', c.latest_message_received_time_us
//           )
//         ) as channels
//         FROM servers s
//         INNER JOIN channel_view c ON c.server = s.uri
//         GROUP BY s.uri, s.name, s.creator, c.uri, c.name

// `;

In [ ]:
import { assert, assertEquals } from "asserts";
import { TestDatabase } from "tinychat/db.ts";

Deno.test("message seeding", () => {
  const tdb = TestDatabase.setup();
  assert(
    tdb.db.prepare("SELECT COUNT(*) FROM messages").value<[number]>()![0] > 80,
    "messaging seeding works",
  );
});

In [ ]:
Deno.test("message cursor", () => {
  const t = `${new Date().getTime() * 1000}`;
  const cursor = MessageCursor.fromString(
    new MessageCursor(t, "past").toString(),
  );
  assert(cursor.timestamp === t, "timestamp matches");
  assert(cursor.direction === "past", "direction matches");
});

In [ ]:
Deno.test("message loading and pagination with default latest sorting", async (t) => {
  const tdb = TestDatabase.setup({ messages: false });
  const messaging = new Messaging(tdb.db);
  const db = tdb.db;

  //throw a bunch of messages into the db
  for (let i = 0; i < 1000; i++) {
    // offset ts by i minutes
    const timestamp = `${(new Date().getTime() + 60 * (i * 1000)) * 1000}`;
    tdb.user1MessagesChannel1(`[${i}] hello world`, timestamp);
  }

  await t.step("test get by uri", () => {
    const uri =
      db.prepare("SELECT uri FROM messages").get<{ uri: string }>()!.uri;
    const { messages } = messaging.getMessages({ uri });
    assertEquals(messages.length, 1, "got 1 message");
    assertEquals(messages[0].uri, uri, "got the right message");
  });

  await t.step("test pagination and order for the first batch", () => {
    const { messages, prevCursor, nextCursor } = messaging.getMessages({
      server: TestDatabase.server,
      channel: TestDatabase.channel1,
      limit: 10,
    });

    assertEquals(messages.length, 10, "got 10 messages");
    assertEquals(
      messages[0].text,
      "[999] hello world",
      "the latest message comes first",
    );
    assertEquals(
      messages[9].text,
      "[990] hello world",
      "the oldest message comes last",
    );
    assert(messages[0].text > messages[9].text, "latest messages come first");

    assert(
      !nextCursor,
      "no next cursor for the first fetch with the latest messages",
    );
    assert(prevCursor, "got a prev cursor for the first fetch");
    assert(
      MessageCursor.fromString(prevCursor!).timestamp === messages[9].ts,
      "prev cursor points to the oldest in the batch",
    );
    assert(
      MessageCursor.fromString(prevCursor!).direction === "past",
      "prev cursor points to the past",
    );
  });

  await t.step(
    "test pagination and order for the second batch going into past",
    () => {
      const firstBatch = messaging.getMessages({
        server: TestDatabase.server,
        channel: TestDatabase.channel1,
        limit: 10,
      });
      const { messages, nextCursor, prevCursor } = messaging.getMessages({
        server: TestDatabase.server,
        channel: TestDatabase.channel1,
        limit: 10,
        cursor: firstBatch.prevCursor,
      });

      assertEquals(messages.length, 10, "past batch has 10 messages");
      assert(nextCursor, "past batch has next cursor");
      assert(prevCursor, "past batch has prev cursor");
      assert(
        Number(messages[0].ts) <
          Number(firstBatch.messages[firstBatch.messages.length - 1].ts),
        "past batch latest messages is older than the first batch's last message",
      );
      assert(
        MessageCursor.fromString(nextCursor).timestamp === messages[0].ts,
        "past batch next cursor points to its newest message",
      );
      assert(
        MessageCursor.fromString(nextCursor).direction === "future",
        "past batch next cursor points to the future",
      );
      assert(
        MessageCursor.fromString(prevCursor).timestamp ===
          messages[messages.length - 1].ts,
        "past batch prev cursor points to its oldest message",
      );
      assert(
        MessageCursor.fromString(prevCursor).direction === "past",
        "past batch prev cursor points to the past",
      );
    },
  );

  await t.step(
    "test pagination and order going forward from the second batch",
    () => {
      const firstBatch = messaging.getMessages({
        server: TestDatabase.server,
        channel: TestDatabase.channel1,
        limit: 10,
      });
      const secondBatch = messaging.getMessages({
        server: TestDatabase.server,
        channel: TestDatabase.channel1,
        limit: 10,
        cursor: firstBatch.prevCursor,
      });
      const { messages, nextCursor, prevCursor } = messaging.getMessages({
        server: TestDatabase.server,
        channel: TestDatabase.channel1,
        limit: 10,
        cursor: secondBatch.nextCursor,
      });

      assertEquals(messages.length, 10, "got 10 messages");
      assert(
        messages[0].ts > messages[messages.length - 1].ts,
        "new messages comes first",
      );
      assert(prevCursor, "got prev cursor for the new batch");
      assert(nextCursor, "got next cursor for the new batch");
    },
  );
});

In [ ]:
Deno.test(
  "message loading and pagination with chronological sorting",
  async (t) => {
    const tdb = TestDatabase.setup({ messages: false });
    const messaging = new Messaging(tdb.db);

    //throw a bunch of messages into the db
    for (let i = 0; i < 1000; i++) {
      // offset ts by i minutes
      const timestamp = `${(new Date().getTime() + 60 * (i * 1000)) * 1000}`;
      tdb.user1MessagesChannel1(`[${i}] hello world`, timestamp);
    }

    await t.step("test pagination and order for the first batch", () => {
      const { messages, prevCursor, nextCursor } = messaging.getMessages({
        server: TestDatabase.server,
        channel: TestDatabase.channel1,
        limit: 10,
        sort: "chronological",
      });

      assertEquals(messages.length, 10, "got 10 messages");
      assertEquals(
        messages[0].text,
        "[0] hello world",
        "first message comes first",
      );
      assertEquals(
        messages[9].text,
        "[9] hello world",
        "later message comes last",
      );

      assert(nextCursor, "got next cursor for the first batch");
      assert(!prevCursor, "no previous cursor");
      assert(
        MessageCursor.fromString(nextCursor!).timestamp === messages[9].ts,
        "next cursor points to the newest message in the batch",
      );
      assert(
        MessageCursor.fromString(nextCursor!).direction === "future",
        "next cursor points to the future",
      );
    });

    await t.step(
      "test pagination and order for the second batch going into future",
      () => {
        const firstBatch = messaging.getMessages({
          server: TestDatabase.server,
          channel: TestDatabase.channel1,
          limit: 10,
          sort: "chronological",
        });
        const { messages, nextCursor, prevCursor } = messaging.getMessages({
          server: TestDatabase.server,
          channel: TestDatabase.channel1,
          limit: 10,
          cursor: firstBatch.nextCursor,
          sort: "chronological",
        });

        assertEquals(messages.length, 10, "future batch has 10 messages");
        assert(nextCursor, "future batch has next cursor");
        assert(prevCursor, "future batch has prev cursor");
        assert(
          Number(messages[0].ts) >
            Number(firstBatch.messages[firstBatch.messages.length - 1].ts),
          "future batch first message is newer than the last message of the previous batch",
        );
        assert(
          MessageCursor.fromString(prevCursor).timestamp === messages[0].ts,
          "future batch prev cursor points to its first message",
        );
        assert(
          MessageCursor.fromString(nextCursor).direction === "future",
          "future batch next cursor points to the future",
        );
        assert(
          MessageCursor.fromString(prevCursor).timestamp === messages[0].ts,
          "future batch prev cursor points to its oldest message",
        );
        assert(
          MessageCursor.fromString(prevCursor).direction === "past",
          "future batch prev cursor points to the past",
        );
      },
    );

    await t.step(
      "test pagination and order going backwards from the second batch",
      () => {
        const firstBatch = messaging.getMessages({
          server: TestDatabase.server,
          channel: TestDatabase.channel1,
          limit: 10,
          sort: "chronological",
        });
        const secondBatch = messaging.getMessages({
          server: TestDatabase.server,
          channel: TestDatabase.channel1,
          limit: 10,
          cursor: firstBatch.nextCursor,
          sort: "chronological",
        });

        // back to the first batch
        const { messages, nextCursor, prevCursor } = messaging.getMessages({
          server: TestDatabase.server,
          channel: TestDatabase.channel1,
          limit: 10,
          cursor: secondBatch.prevCursor,
          sort: "chronological",
        });

        assertEquals(messages.length, 10, "got 10 messages");
        assertEquals(
          messages[0].text,
          "[0] hello world",
          "got the first message",
        );
        assert(
          messages[0].ts < messages[messages.length - 1].ts,
          "odlers messages comes first",
        );
        assert(!prevCursor, "there is no prev cursor for the new batch");
        assert(nextCursor, "got next cursor for the new batch");
      },
    );
  },
);

In [ ]:
Deno.test("delete message", () => {
  const tdb = TestDatabase.setup();
  const messaging = new Messaging(tdb.db);
  const m = tdb.db.prepare("SELECT * FROM messages").get<{ uri: string }>();

  messaging.deleteMessage({
    did: TestDatabase.user1,
    time_us: 1738269878791634,
    commit: {
      rev: "3lgydzdyarc2b",
      operation: "delete",
      collection: "chat.tinychat.core.server",
      rkey: "k",
    },
    uri: m!.uri,
  });

  const deletedMessage = tdb.db
    .prepare(`SELECT * FROM messages WHERE uri = '${m!.uri}'`)
    .get<{ deleted_at: string, text: string }>();

  assert(deletedMessage!.deleted_at, "message has deleted timestamp set");
  assertEquals(deletedMessage!.text, "<deleted>", "message text is deleted");
});